In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
                    .master('local[1]') \
                    .appName('xinxuequotaapp') \
                    .getOrCreate()

StatementMeta(, 476e613f-7f51-4154-96c0-12aa7660b603, 3, Finished, Available)

In [2]:
%pip install yfinance

import yfinance as yf

ticker = ticker = ["MSFT", "AMZN", "META", "AAPL", "GOOGL", "NVDA", "TSLA"] 
stock_prices_data = yf.download(ticker, period='1y')

StatementMeta(, 476e613f-7f51-4154-96c0-12aa7660b603, 9, Finished, Available)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.0/73.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.7/116.7 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 138.0 MB/s eta 0:00:00
  Installing build dependencies ... - \ | done
  Getting requirements to build wheel ... - \ | / done
  Preparing metadata (pyproject.toml) ... - \ done
  Created wheel for peewee: filename=peewee-3.17.1-cp310-cp310-linux_x86_64.whl size=298267 sha256=131660c5cc80fbdbc22d8e767950f850009f673e5ddc3dae5ffa9427bbaafc68
  Stored in directory: /home/trusted-service-user/.cache/pip/wheels/d7/35/5c/1374782be033462df5f40174d8d879519d64ed8c25a1977554
Successfully built peewee

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



[*********************100%%**********************]  7 of 7 completed


In [3]:
stock_prices=stock_prices_data["Adj Close"]
print(stock_prices)

StatementMeta(, 476e613f-7f51-4154-96c0-12aa7660b603, 11, Finished, Available)

Ticker            AAPL        AMZN       GOOGL        META        MSFT  \
Date                                                                     
2023-03-21  158.434311  100.610001  104.919998  201.945740  271.551300   
2023-03-22  156.992020   98.699997  103.370003  199.598221  270.073456   
2023-03-23  158.086166   98.709999  105.599998  204.063492  275.399719   
2023-03-24  159.399185   98.129997  105.440002  205.791641  278.286041   
2023-03-27  157.439636   98.040001  102.459999  202.625015  274.130157   
...                ...         ...         ...         ...         ...   
2024-03-14  173.000000  178.750000  143.100006  491.829987  425.220001   
2024-03-15  172.619995  174.419998  141.179993  484.100006  416.420013   
2024-03-18  173.720001  174.479996  147.679993  496.980011  417.320007   
2024-03-19  176.080002  175.899994  147.029999  496.239990  421.410004   
2024-03-20  178.669998  178.149994  148.740005  505.519989  425.230011   

Ticker            NVDA        TSLA  


In [4]:
stock_prices.reset_index(drop=False, inplace=True)
df_spark = spark.createDataFrame(stock_prices)

StatementMeta(, 476e613f-7f51-4154-96c0-12aa7660b603, 12, Finished, Available)

In [5]:
print(df_spark.head(2))
print(df_spark.tail(2))

StatementMeta(, 476e613f-7f51-4154-96c0-12aa7660b603, 13, Finished, Available)

[Row(Date=datetime.datetime(2023, 3, 21, 0, 0), AAPL=158.43431091308594, AMZN=100.61000061035156, GOOGL=104.91999816894531, META=201.94573974609375, MSFT=271.5513000488281, NVDA=261.9059753417969, TSLA=197.5800018310547), Row(Date=datetime.datetime(2023, 3, 22, 0, 0), AAPL=156.9920196533203, AMZN=98.69999694824219, GOOGL=103.37000274658203, META=199.5982208251953, MSFT=270.0734558105469, NVDA=264.59515380859375, TSLA=191.14999389648438)]
[Row(Date=datetime.datetime(2024, 3, 19, 0, 0), AAPL=176.0800018310547, AMZN=175.89999389648438, GOOGL=147.02999877929688, META=496.239990234375, MSFT=421.4100036621094, NVDA=893.97998046875, TSLA=171.32000732421875), Row(Date=datetime.datetime(2024, 3, 20, 0, 0), AAPL=178.6699981689453, AMZN=178.14999389648438, GOOGL=148.74000549316406, META=505.5199890136719, MSFT=425.2300109863281, NVDA=903.719970703125, TSLA=175.66000366210938)]


In [6]:
from pyspark.sql.types import *
from delta.tables import DeltaTable

spark.sql("DROP TABLE IF EXISTS quota")

dt1 = DeltaTable.createIfNotExists(spark)\
.tableName("quota")\
.addColumn("Date", dataType="TIMESTAMP", nullable=True)\
.addColumn("AAPL", dataType="Double", nullable=True)\
.addColumn("AMZN", dataType="Double", nullable=True)\
.addColumn("GOOGL", dataType="Double", nullable=True)\
.addColumn("META", dataType="Double", nullable=True)\
.addColumn("MSFT", dataType="Double", nullable=True)\
.addColumn("NVDA", dataType="Double", nullable=True)\
.addColumn("TSLA", dataType="Double", nullable=True)\
.partitionedBy("Date")\
.execute()


StatementMeta(, 476e613f-7f51-4154-96c0-12aa7660b603, 14, Finished, Available)

In [7]:
df_spark.write.format("delta").mode("overwrite").saveAsTable("quota")
print("done...")

StatementMeta(, 476e613f-7f51-4154-96c0-12aa7660b603, 15, Finished, Available)

done...
